# AutogradTensor Development

We have an issue with backprop in PyTorch. When you call `.backward()` on a torch.Tensor it immediately drops down into C++ code. However, we need to keep track of all operations in Python for use in calculating sensitivity as well as other things. We need to override `.backward()` such that we go through the backpropagation chain in Python instead of the C++ layer.

The idea here is we create a `BackpropTensor` that overrides `.backward()`. Maybe a different name, like `GradientTensor` since it should only be used on tensors where we need gradients. We can discuss naming later I suppose. Anyway, this new tensor should override `.backward()`...

In [1]:
import sys
sys.path.append('/Users/mat/Projects/PySyft')

# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.tensors.interpreters import AutogradTensor
import torch
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [2]:
mat = sy.VirtualWorker(hook, id="matalicious", verbose=True)

In [3]:
a = torch.tensor([3, 2., 0], requires_grad=True)
b = torch.tensor([1, 2., 3], requires_grad=True)

a = a.send(mat, local_autograd=True, preinitialize_grad=False)
b = b.send(mat, local_autograd=True, preinitialize_grad=False)

worker <VirtualWorker id:matalicious #tensors:0> received OBJ tensor([3., 2., 0.], requires_grad=True)
{}


AttributeError: 'NoneType' object has no attribute 'child'

In [4]:
a

(Wrapper)>AutogradTensor>[PointerTensor | me:69163178181 -> matalicious:81041938722]

In [5]:
a.child.preinitialize_grad

True

In [7]:
c = a * b

worker <VirtualWorker id:matalicious #tensors:4> received CMD ((b'mul', tensor([3., 2., 0.], requires_grad=True), (tensor([1., 2., 3.], requires_grad=True),), {}), [16856263657])
{}
worker <VirtualWorker id:matalicious #tensors:5> received CMD ((b'__setattr__', tensor([3., 4., 0.], grad_fn=<MulBackward0>), (b'grad', None), {}), [53107197043])
worker <VirtualWorker id:matalicious #tensors:5> received OBJ_DEL 53107197043


In [16]:
a.grad.get()

worker <VirtualWorker id:matalicious #tensors:8> received OBJ_REQ 32398144356


tensor([0., 0., 0.], requires_grad=True)

In [12]:
c.child.preinitialize_grad

False

In [8]:
c.backward(torch.tensor([1., 1, 1]).send(mat))

worker <VirtualWorker id:matalicious #tensors:5> received OBJ tensor([1., 1., 1.])
worker <VirtualWorker id:matalicious #tensors:6> received CMD ((b'__mul__', tensor([1., 1., 1.]), (tensor([1., 2., 3.], requires_grad=True),), {}), [16953201404])
worker <VirtualWorker id:matalicious #tensors:7> received CMD ((b'__mul__', tensor([1., 1., 1.]), (tensor([3., 2., 0.], requires_grad=True),), {}), [55476961395])


TensorsNotCollocatedException: You tried to call a method involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:matalicious #tensors:8> while the other is on <VirtualWorker id:matalicious #tensors:8>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [7]:
a

(Wrapper)>AutogradTensor>[PointerTensor | me:74891028609 -> matalicious:35296027662]

In [10]:
a.grad.get()

worker <VirtualWorker id:matalicious #tensors:5> received OBJ_REQ 48031574094
worker <VirtualWorker id:matalicious #tensors:4> received OBJ_DEL 48031574094


tensor([0., 0., 0.], requires_grad=True)

In [1]:
import sys
sys.path.append('/Users/mat/Projects/PySyft')

# Run this cell to see if things work
import syft as sy
from syft.frameworks.torch.tensors.interpreters import AutogradTensor
import torch
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

In [2]:
a = AutogradTensor().on(torch.tensor([3, 2., 0]))
b = AutogradTensor().on(torch.tensor([1, 2., 3]))
c = a + b

In [3]:
c.grad_fn(torch.ones(3))

(tensor([1., 1., 1.]), tensor([1., 1., 1.]))

In [4]:
c.backward()

In [5]:
a

(Wrapper)>AutogradTensor>tensor([3., 2., 0.])

In [6]:
# need to fix this, should show the gradient
a.grad

In [7]:
a.child.grad

AutogradTensor>tensor([1., 1., 1.])

In [8]:
b.child.grad

tensor([1., 1., 1.])

In [9]:
d = AutogradTensor().on(torch.rand(3))
e = AutogradTensor().on(torch.rand(3))

In [10]:
f = d.sin() * e.sin()

In [11]:
f.grad_fn

MulBackward

In [12]:
f.grad_fn.next_functions

(SinBackward, SinBackward)

In [13]:
f.backward()

In [15]:
e.child.grad

tensor([0.4016, 0.0379, 0.1268])